# Product Owner LoRA Model - Inference Testing

Este notebook prueba el modelo Product Owner entrenado con LoRA.

**Requisitos**:
- Google Colab con GPU (T4 Free)
- **Opción A**: Modelo en Google Drive: `/content/drive/MyDrive/colab_models/po_student_v1`
- **Opción B**: El notebook clonará automáticamente el repositorio si el modelo no está en Drive

**Pasos**:
1. Verificar GPU
2. Instalar dependencias
3. Montar Google Drive (opcional) o clonar repositorio
4. Probar modelo baseline (sin LoRA)
5. Probar modelo fine-tuned (con LoRA)
6. Comparar resultados

## 1. Verificar GPU

In [ ]:
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 2. Instalar Dependencias

In [ ]:
# NO reinstalar PyTorch - Colab ya tiene PyTorch con CUDA 12.x
!pip install -q transformers>=4.40.0 peft>=0.10.0 bitsandbytes>=0.43.0 accelerate>=0.28.0

# Verificar instalación
import transformers
import peft
import bitsandbytes
import accelerate

print("✅ All packages installed successfully")
print(f"transformers: {transformers.__version__}")
print(f"peft: {peft.__version__}")
print(f"bitsandbytes: {bitsandbytes.__version__}")
print(f"accelerate: {accelerate.__version__}")

## 3. Cargar Modelo LoRA

Busca el modelo en Drive, local, o clona el repositorio automáticamente.

In [ ]:
import os
from pathlib import Path

# 1. Montar Google Drive (opcional - si falla, usamos el repo)
print("🔄 Intentando montar Google Drive...")
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    
    if os.path.exists('/content/drive/MyDrive'):
        print("✅ Google Drive montado correctamente\n")
        drive_available = True
    else:
        print("⚠️  Drive montado pero MyDrive no accesible\n")
        drive_available = False
except Exception as e:
    print(f"⚠️  No se pudo montar Drive: {e}")
    print("📦 Usaremos el modelo desde el repositorio\n")
    drive_available = False

# 2. Buscar el modelo en ubicaciones posibles
print("🔍 Buscando modelo LoRA entrenado...")

possible_locations = [
    "/content/drive/MyDrive/colab_models/po_student_v1",  # Drive principal
    "/content/drive/MyDrive/po_student_v1",               # Drive alternativo
    "/content/po_student_v1",                              # Local en Colab
    "/content/agnostic-ai-pipeline/artifacts/models/po_student_v1",  # Repo clonado
]

model_path = None
for path in possible_locations:
    if os.path.exists(path):
        model_path = path
        print(f"✅ Modelo encontrado en: {model_path}")
        break

# 3. Si no se encontró el modelo, clonar el repositorio
if model_path is None:
    print("❌ Modelo no encontrado en Drive ni local")
    print("\n📥 Clonando repositorio con el modelo...")
    
    repo_url = "https://github.com/matiasleandrokruk/agnostic-ai-pipeline.git"
    repo_path = "/content/agnostic-ai-pipeline"
    
    if not os.path.exists(repo_path):
        !git clone --depth 1 {repo_url} {repo_path}
        print(f"✅ Repositorio clonado en: {repo_path}")
    else:
        print(f"✅ Repositorio ya existe en: {repo_path}")
    
    # Verificar si el modelo está en el repo
    model_path = f"{repo_path}/artifacts/models/po_student_v1"
    
    if os.path.exists(model_path):
        print(f"✅ Modelo encontrado en el repositorio: {model_path}")
    else:
        print(f"\n❌ ERROR: Modelo no encontrado ni en Drive ni en el repositorio")
        print("\n📍 Ubicaciones buscadas:")
        for path in possible_locations + [model_path]:
            print(f"  - {path}")
        print("\n💡 Soluciones:")
        print("1. Verifica que el modelo está en el repositorio en: artifacts/models/po_student_v1/")
        print("2. O copia el modelo a Google Drive: /content/drive/MyDrive/colab_models/po_student_v1")
        print("3. O ejecuta primero el training notebook para generar el modelo")
        raise FileNotFoundError("Modelo LoRA no encontrado en ninguna ubicación")

# 4. Verificar archivos críticos del modelo
print(f"\n📂 Contenido del modelo:")
!ls -lh {model_path}

required_files = ["adapter_config.json", "adapter_model.safetensors"]
missing_files = []

for file in required_files:
    file_path = os.path.join(model_path, file)
    if not os.path.exists(file_path):
        missing_files.append(file)
    else:
        file_size = os.path.getsize(file_path) / 1024**2  # MB
        print(f"  ✓ {file} ({file_size:.1f} MB)")

if missing_files:
    print(f"\n⚠️  ADVERTENCIA: Faltan archivos del modelo: {missing_files}")
    print("El modelo podría estar incompleto. Verifica que el training terminó correctamente.")
else:
    print("\n✅ Todos los archivos del modelo están presentes")

# 5. Guardar el path del modelo para usar después
MODEL_PATH = model_path
print(f"\n✅ Listo para usar el modelo desde: {MODEL_PATH}")

## 4. Definir Test Cases

Test cases con prompts realistas de Product Owner:

In [ ]:
TEST_CASES = [
    {
        "name": "basic_blog_validation",
        "prompt": """[INSTRUCTIONS]
You are a Product Owner reviewing requirements from a Business Analyst.
Validate the requirements, identify missing details, and provide feedback.

[REQUIREMENTS]
Title: Simple Blog Platform
Description: Users should be able to create and view blog posts.

Features:
- Users can create blog posts
- Posts have title and content
- Users can view all published posts

[YOUR RESPONSE]""",
    },
    {
        "name": "ecommerce_requirements",
        "prompt": """[INSTRUCTIONS]
You are a Product Owner reviewing requirements from a Business Analyst.
Validate the requirements, identify missing details, and provide feedback.

[REQUIREMENTS]
Title: E-commerce Product Catalog
Description: Build a product catalog for an online store.

Features:
- Display list of products with images
- Show product details (name, price, description)
- Search products by name
- Filter by category

[YOUR RESPONSE]""",
    },
    {
        "name": "incomplete_requirements",
        "prompt": """[INSTRUCTIONS]
You are a Product Owner reviewing requirements from a Business Analyst.
Validate the requirements, identify missing details, and provide feedback.

[REQUIREMENTS]
Title: Social Media Integration
Description: Add social media features.

Features:
- Share posts
- Like content

[YOUR RESPONSE]""",
    },
]

print(f"📝 {len(TEST_CASES)} test cases defined")

## 5. Funciones de Carga y Generación

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

def load_model(base_model_name, adapter_path=None, load_4bit=True):
    """
    Carga el modelo base o base + LoRA adapter.
    
    Args:
        base_model_name: Nombre del modelo base en HuggingFace
        adapter_path: Path al adapter LoRA (None para baseline)
        load_4bit: Usar quantization 4-bit
    
    Returns:
        (tokenizer, model)
    """
    print(f"[load_model] Cargando base model: {base_model_name}...")
    
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    model_kwargs = {"device_map": "auto"}
    
    if load_4bit:
        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )
        model_kwargs.update(
            quantization_config=quant_config,
            torch_dtype=torch.float16,
        )
    else:
        model_kwargs["torch_dtype"] = torch.float16
    
    model = AutoModelForCausalLM.from_pretrained(base_model_name, **model_kwargs)
    
    if adapter_path:
        print(f"[load_model] Cargando LoRA adapter desde: {adapter_path}...")
        model = PeftModel.from_pretrained(model, adapter_path)
        print("[load_model] ✅ LoRA adapter cargado")
    else:
        print("[load_model] Modo baseline (sin adapter)")
    
    return tokenizer, model


def generate_response(
    tokenizer,
    model,
    prompt,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9,
):
    """
    Genera respuesta para un prompt.
    
    Args:
        tokenizer: Tokenizer instance
        model: Model instance
        prompt: Input prompt
        max_new_tokens: Máximo de tokens a generar
        temperature: Temperatura de sampling
        top_p: Nucleus sampling parameter
    
    Returns:
        Generated text
    """
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
    
    # Mover inputs al device del modelo
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # Decodificar y extraer solo la parte generada (skip input prompt)
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Intentar extraer solo la respuesta después del prompt
    if "[YOUR RESPONSE]" in full_text:
        response = full_text.split("[YOUR RESPONSE]", 1)[1].strip()
    else:
        # Fallback: remover el tamaño del prompt original
        response = full_text[len(prompt):].strip()
    
    return response

print("✅ Funciones definidas")

## 6. Probar Modelo Baseline (sin LoRA)

Primero probamos el modelo base Qwen2.5-7B sin fine-tuning:

In [ ]:
BASE_MODEL = "Qwen/Qwen2.5-7B-Instruct"

print("🔄 Cargando modelo BASELINE (sin LoRA)...\n")
try:
    tokenizer_baseline, model_baseline = load_model(BASE_MODEL, adapter_path=None)
except Exception as e:
    print(f"❌ ERROR cargando modelo: {e}")
    print("\nPosibles soluciones:")
    print("1. Verifica que tienes suficiente RAM/VRAM disponible")
    print("2. Reinicia el runtime: Runtime → Restart runtime")
    print("3. Verifica que Runtime type = T4 GPU")
    raise

print("\n" + "="*80)
print("TESTING BASELINE MODEL")
print("="*80 + "\n")

baseline_results = []

for idx, test_case in enumerate(TEST_CASES):
    print(f"\n--- Test Case {idx + 1}/{len(TEST_CASES)}: {test_case['name']} ---\n")
    
    prompt = test_case["prompt"]
    print("[Prompt Preview]")
    print(prompt[:150] + "...")
    
    print("\n[Generando respuesta...]")
    try:
        response = generate_response(tokenizer_baseline, model_baseline, prompt)
        print("\n[Response]")
        print(response)
    except Exception as e:
        print(f"\n❌ ERROR generando respuesta: {e}")
        response = f"[ERROR: {str(e)}]"
    
    print("\n" + "="*80)
    
    baseline_results.append({
        "test_case": test_case["name"],
        "prompt": prompt,
        "response": response,
        "response_length": len(response),
    })

print(f"\n✅ Baseline testing completado: {len(baseline_results)} test cases")

## 7. Liberar Memoria del Modelo Baseline

In [ ]:
# Liberar memoria antes de cargar el modelo fine-tuned
import gc

del model_baseline
del tokenizer_baseline
gc.collect()
torch.cuda.empty_cache()

print("✅ Memoria liberada")

## 8. Probar Modelo Fine-tuned (con LoRA)

Ahora probamos el modelo con el adapter LoRA entrenado:

In [ ]:
BASE_MODEL = "Qwen/Qwen2.5-7B-Instruct"

# Usar la variable MODEL_PATH definida en la celda de montaje de Drive
# Si no existe (porque ejecutaste esta celda directamente), usar path por defecto
try:
    ADAPTER_PATH = MODEL_PATH
    print(f"✅ Usando modelo desde: {ADAPTER_PATH}")
except NameError:
    ADAPTER_PATH = "/content/drive/MyDrive/colab_models/po_student_v1"
    print(f"⚠️  MODEL_PATH no definido, usando path por defecto: {ADAPTER_PATH}")
    if not os.path.exists(ADAPTER_PATH):
        raise FileNotFoundError(f"Modelo no encontrado en {ADAPTER_PATH}. Ejecuta primero la celda 3 (Montar Google Drive)")

print("\n🔄 Cargando modelo FINE-TUNED (con LoRA)...\n")
tokenizer_ft, model_ft = load_model(BASE_MODEL, adapter_path=ADAPTER_PATH)

print("\n" + "="*80)
print("TESTING FINE-TUNED MODEL")
print("="*80 + "\n")

finetuned_results = []

for idx, test_case in enumerate(TEST_CASES):
    print(f"\n--- Test Case {idx + 1}/{len(TEST_CASES)}: {test_case['name']} ---\n")
    
    prompt = test_case["prompt"]
    print("[Prompt Preview]")
    print(prompt[:150] + "...")
    
    print("\n[Generando respuesta...]")
    try:
        response = generate_response(tokenizer_ft, model_ft, prompt)
        print("\n[Response]")
        print(response)
    except Exception as e:
        print(f"\n❌ ERROR generando respuesta: {e}")
        response = f"[ERROR: {str(e)}]"
    
    print("\n" + "="*80)
    
    finetuned_results.append({
        "test_case": test_case["name"],
        "prompt": prompt,
        "response": response,
        "response_length": len(response),
    })

print(f"\n✅ Fine-tuned testing completado: {len(finetuned_results)} test cases")

## 9. Comparación de Resultados

Comparamos las respuestas de ambos modelos:

In [ ]:
import json

print("\n" + "="*80)
print("COMPARISON SUMMARY")
print("="*80 + "\n")

for idx, (base_result, ft_result) in enumerate(zip(baseline_results, finetuned_results)):
    print(f"\n### Test Case {idx + 1}: {base_result['test_case']}")
    print("\n#### Baseline Model:")
    print(base_result["response"][:200] + "..." if len(base_result["response"]) > 200 else base_result["response"])
    print(f"\nLength: {base_result['response_length']} chars")
    
    print("\n#### Fine-tuned Model:")
    print(ft_result["response"][:200] + "..." if len(ft_result["response"]) > 200 else ft_result["response"])
    print(f"\nLength: {ft_result['response_length']} chars")
    
    print("\n" + "-"*80)

# Estadísticas
avg_baseline_length = sum(r["response_length"] for r in baseline_results) / len(baseline_results)
avg_ft_length = sum(r["response_length"] for r in finetuned_results) / len(finetuned_results)

print("\n" + "="*80)
print("STATISTICS")
print("="*80)
print(f"Total test cases: {len(baseline_results)}")
print(f"Average baseline response length: {avg_baseline_length:.0f} chars")
print(f"Average fine-tuned response length: {avg_ft_length:.0f} chars")
print(f"Difference: {avg_ft_length - avg_baseline_length:+.0f} chars ({(avg_ft_length / avg_baseline_length - 1) * 100:+.1f}%)")

## 10. Guardar Resultados

In [ ]:
import json
from datetime import datetime

# Guardar resultados en Drive
output_dir = "/content/drive/MyDrive/colab_models/inference_results"
!mkdir -p {output_dir}

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Baseline results
baseline_file = f"{output_dir}/baseline_{timestamp}.json"
with open(baseline_file, "w") as f:
    json.dump({
        "model": BASE_MODEL,
        "adapter_path": None,
        "timestamp": timestamp,
        "results": baseline_results,
    }, f, indent=2, ensure_ascii=False)

print(f"✅ Baseline results saved to: {baseline_file}")

# Fine-tuned results
finetuned_file = f"{output_dir}/finetuned_{timestamp}.json"
with open(finetuned_file, "w") as f:
    json.dump({
        "model": BASE_MODEL,
        "adapter_path": ADAPTER_PATH,
        "timestamp": timestamp,
        "results": finetuned_results,
    }, f, indent=2, ensure_ascii=False)

print(f"✅ Fine-tuned results saved to: {finetuned_file}")

# Comparison summary
comparison_file = f"{output_dir}/comparison_{timestamp}.json"
with open(comparison_file, "w") as f:
    json.dump({
        "timestamp": timestamp,
        "baseline_avg_length": avg_baseline_length,
        "finetuned_avg_length": avg_ft_length,
        "difference": avg_ft_length - avg_baseline_length,
        "difference_percent": (avg_ft_length / avg_baseline_length - 1) * 100,
        "test_cases": len(baseline_results),
    }, f, indent=2)

print(f"✅ Comparison summary saved to: {comparison_file}")
print("\n" + "="*80)
print("✅ TESTING COMPLETO")
print("="*80)